In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

C:\Users\serda\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\serda\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\serda\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
  stacklevel=1)


# Read Preprocessed Data

In [3]:
data = pd.read_csv(r'Data\preprocessed_data.csv')

In [4]:
x_data = data["text"][pd.isna(data["text"])==False]
y_data = data["target"][pd.isna(data["text"])== False]

In [5]:
x_data

0                           haha you guy are bunch of loser
1                                         ur sh tty comment
2         it ridiculous that these guy are being called ...
3         this story get more ridiculous by the hour and...
4         angry troll misogynist and racist oh my it doe...
5         yet call out all muslim for the act of few wil...
6            this bitch is nut who would read book by woman
7         tuesday jan 13 member of this socalled militia...
8         love that they also participate in the portlan...
9         took this a an opportunity to check back in on...
10        their ridiculous band photo ha me wanting to d...
11        will whiteness history month make all white pe...
12        anybody who can kiss that many ass that quickl...
13        metro ha lied to u from the beginning about wh...
14        on it face glueck claim that ag rosenblum need...
15        um bernie supporter attack thom hartmann liar ...
16                                      

# Bag of Word Model

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

In [6]:
%%time
vectorizer = CountVectorizer(max_features=512, min_df=5, max_df=0.95, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(x_data).toarray() 

Wall time: 9.6 s


In [7]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

# Train Test Split

In [11]:
from sklearn.model_selection import train_test_split

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X, y_data, test_size=0.2, random_state=32)

In [10]:
np.unique(y_test, return_counts=True)

(array([0, 1], dtype=int64), array([31649, 32198], dtype=int64))

# Sklearn

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

In [14]:
%%time
model = AdaBoostClassifier()
model.fit(x_train, y_train)
model.score(x_test, y_test)

Wall time: 3min 7s


0.7466913089103638

In [15]:
%%time
model = DecisionTreeClassifier()
model.fit(x_train, y_train)
model.score(x_test, y_test)

Wall time: 3min 58s


0.6917317963255909

In [16]:
%%time
model = RandomForestClassifier()
model.fit(x_train, y_train)
model.score(x_test, y_test)

Wall time: 11min 42s


0.7571381584099488

%%time
model = SVC()
model.fit(x_train, y_train)
model.score(x_test, y_test)

%%time
model = KNeighborsClassifier()
model.fit(x_train, y_train)
model.score(x_test, y_test)

In [17]:
%%time
model = GaussianNB()
model.fit(x_train, y_train)
model.score(x_test, y_test)

Wall time: 2.03 s


0.6906510877566683

# XGBOOST

In [18]:
import xgboost

C:\Users\serda\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [31]:
%%time
model = xgboost.XGBClassifier(n_estimators=100 ,n_jobs=6,max_depth=25, tree_method='gpu_hist')
model.fit(x_train, y_train )
model.score(x_test, y_test)

Wall time: 1min 10s


0.7617272542171127

## XGBoost Parameter Tuning

In [15]:
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.95, stop_words=stopwords.words('english'))

In [16]:
X = vectorizer.fit_transform(x_data).toarray() 

In [17]:
x_train, x_test, y_train, y_test = train_test_split(X, y_data, test_size=0.2, random_state=32)

In [21]:
model = xgboost.XGBClassifier(n_estimators=100 ,n_jobs=6,max_depth=25, tree_method='gpu_hist')
model.fit(x_train, y_train )
model.score(x_test, y_test)

[11:14:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.8265071185803562

In [22]:
model = xgboost.XGBClassifier(n_estimators=200 ,n_jobs=6,max_depth=25, tree_method='gpu_hist')
model.fit(x_train, y_train )
model.score(x_test, y_test)

[11:18:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.8284022741867276

In [23]:
model = xgboost.XGBClassifier(n_estimators=200 ,n_jobs=6,max_depth=20, tree_method='gpu_hist')
model.fit(x_train, y_train )
model.score(x_test, y_test)

[11:24:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.8297492442871239

In [24]:
model = xgboost.XGBClassifier(n_estimators=250 ,n_jobs=6,max_depth=20, tree_method='gpu_hist')
model.fit(x_train, y_train )
model.score(x_test, y_test)

[11:27:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.8290131094648143

In [25]:
model = xgboost.XGBClassifier(n_estimators=180 ,n_jobs=6,max_depth=20, tree_method='gpu_hist')
model.fit(x_train, y_train )
model.score(x_test, y_test)

[11:31:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.8286058859460899

In [26]:
model = xgboost.XGBClassifier(n_estimators=180 ,n_jobs=6,max_depth=16, tree_method='gpu_hist')
model.fit(x_train, y_train )
model.score(x_test, y_test)

[11:34:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.8273685529468886

In [27]:
model = xgboost.XGBClassifier(n_estimators=160 ,n_jobs=6,max_depth=20, tree_method='gpu_hist')
model.fit(x_train, y_train )
model.score(x_test, y_test)

[11:36:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.8284962488448948

In [28]:
model = xgboost.XGBClassifier(n_estimators=160 ,n_jobs=6,max_depth=16, tree_method='gpu_hist')
model.fit(x_train, y_train )
model.score(x_test, y_test)

[11:38:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.8273059031747771